# Text Generation with RNNs
https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

### 1 Dataset
Define the path of the file, you want to read and train the model on


In [2]:
'''TODO: set the path of the file'''
path_to_file = 'Shakespeare_karpathy.txt'
text = open(path_to_file, encoding='utf-8').read()


#### Inspect the dataset
Take a look at the first 250 characters in text

In [3]:
print(text[:250])

That, poor contempt, or claim'd thou slept so faithful,
I may contrive our father; and, in their defeated queen,
Her flesh broke me and puttance of expedition house,
And in that same that ever I lament this stomach,
And he, nor Butly and my fury, kno


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

62 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [6]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [7]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'That, poor co' ---- characters mapped to int ---- > [29 43 36 55  4  1 51 50 50 53  1 38 50]


#### Defining a method to encode one hot labels

In [8]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot


#### Defining a method to make mini-batches for training

In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr) // batch_size_total
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n + seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n + seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [10]:
train_on_gpu = torch.cuda.is_available()
print ('Training on GPU' if train_on_gpu else 'Training on CPU')

Training on GPU



### Declaring the model

In [11]:
class VanillaCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        self.rnn = nn.RNN(len(self.vocab), self.n_hidden, self.n_layers, batch_first=True, dropout=self.drop_prob)   #batch_first=True -> input and output tensors are provided as (batch, seq, feature)
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))

    def forward(self, x, hidden):

        # Passing in the input and hidden state into the model and obtaining outputs
        # x of shape (seq_len, batch, input_size)
        # hidden of shape (num_layers * num_directions, batch, hidden_size)
        out, hidden_t = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden = torch.zeros(self.n_layers, batch_size, self.n_hidden)

        return hidden

In [12]:
class LSTMCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        self.lstm = nn.LSTM(len(self.vocab), self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))


    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden_state = torch.zeros(self.n_layers, batch_size, self.n_hidden).cuda()
        cell_state = torch.zeros(self.n_layers, batch_size, self.n_hidden).cuda()
        # cell state should be included as for now the model only backpropagates the loss over the hidden states and not the cell states
        return (hidden_state, cell_state)



#### Declaring the train method


train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

In [13]:
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if (train_on_gpu):
        model.cuda()

    counter = 0
    n_vocab = len(model.vocab)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        '''TODO: use the get_batches function to generate sequences of the desired size'''
        dataset = get_batches(data, batch_size, seq_length)

        for x, y in dataset:
            counter += 1
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_vocab)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            if type(h) is tuple:
                if train_on_gpu:
                    h = tuple([each.data.cuda() for each in h])
                else:
                    h = tuple([each.data for each in h])
            else:
                if train_on_gpu:
                    h = h.data.cuda()
                else:
                    h = h.data
            # zero accumulated gradients
            model.zero_grad()
            output, h = model(inputs, h) 
            loss = criterion(output, targets.view(-1).long())
            
            # perform backprop
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_vocab)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    if type(val_h) is tuple:
                        if train_on_gpu:
                            val_h = tuple([each.data.cuda() for each in val_h])
                        else:
                            val_h = tuple([each.data for each in val_h])
                    else:
                        if train_on_gpu:
                            val_h = val_h.data.cuda()
                        else:
                            val_h = val_h.data

                    inputs, targets = x, y
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    output, val_h = model(inputs, val_h)

                    val_loss = criterion(output, targets.view(-1).long())

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e + 1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                input_eval = 'Dear'
                print(sample(model, 1000, prime=input_eval, top_k=10))
                
                model.train()  # reset to train mode after iterationg through validation data


##### Defining a method to generate the next character

In [14]:
def predict(model, char, h=None, top_k=None):
    ''' 
    Given a character, predict the next character.
    Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)
    
    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    if type(h) is tuple:
        if train_on_gpu:
            h = tuple([each.data.cuda() for each in h])
        else:
            h = tuple([each.data for each in h])
    else:
        if train_on_gpu:
            h = h.data.cuda()
        else:
            h = h.data
    output, h = model(inputs, h)

    # get the character probabilities
    p = F.softmax(output, dim=1).data
    if (train_on_gpu):
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h



#### Declaring a method to generate new text

In [15]:
def sample(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
        char, h = predict(model, char, h, top_k=top_k)
        chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [16]:
''''TODO: Try changing the number of units in the network to see how it affects performance'''
n_hidden = 256
n_layers = 2

vanilla_model = VanillaCharRNN(vocab, n_hidden, n_layers)
print(vanilla_model)
lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)
print(lstm_model)

VanillaCharRNN(
  (rnn): RNN(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)
LSTMCharRNN(
  (lstm): LSTM(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)



###### Declaring the hyperparameters

In [42]:
''''TODO: Try changing the hyperparameters in the network to see how it affects performance'''
batch_size = 20
seq_length = 20
n_epochs = 20  # start smaller if you are just testing initial behavior


##### Train the model and have fun with the generated texts

In [43]:
train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/20... Step: 50... Loss: 1.5811... Val Loss: 1.7445
Deard though a hund and thou thou are that the wated and we not of my such of you, that you hapted and them on bit wan your contempther so thou be blind, I'll be doet and the wone.

CINDO:
Why! for than the most did the
kan the wost: I hote
Should neporester, and I hoph be so, and beganch of heard without thy desure all,
To shall, I see are.

ANGE VITANIO:
Pars, went for you not of dors and thee,--
Or her death if an I have to the party,
Or my hear;
My lord; he, longem,
When I wome but my gather, if I would not his donathout thy hour twere and broven of even him pent, if enot have with our here, had as their furn and of donguess'd be the head is head.

PEBGLOO:
Hall; live one had of it without be not these more
Thar maje ut is sold,
That have lord, a come in.

TRICIP:
But a gast:
But the sport,
Wherefore.

ANGERUS:
Hav, and me to peed friends as I am your sle's them do prain, and her lied thought time,
And though your ground, 

Epoch: 3/20... Step: 450... Loss: 1.5385... Val Loss: 1.7413
Dear:
How from stear to their full not so wisticther by these than and the weace this say he day!

Second I have his
speak?
I this a hilm'd it some that I smsteach be not the peary hathing and dead offen'd of son, if he stonal hat dight, the that same
Thee at stran.

EMILIA:
Thou art
Till again
Word of you as in wlarery of all.

HASD OF DOIIUS:
O whenger
Than I have then you all for shall bay as from statised had a fatured the wiret my since, take her fatch,
The shall not thee are my good so and a man!

MARTIPF:
Ile though your that true.

AJTORD OF YORKAR:
And who with makes the hud worse,
Is my part for thee still her good estreat with my botts as solf as your be place the pray. Why fouth, and three faline, to me you,
Honours as
Forster flieve my lords aff, my lord:
Bets, I would terdort.

VELLOAS:
So profiors,
Shak in a fame they lord.

KING LONNTALO:
Sirral,
Of I would as intted's the pastun made shouldsted as I lack'd a 

Epoch: 4/20... Step: 850... Loss: 1.6156... Val Loss: 1.7391
Dear amms, see with the brother
Them took a suad a would turn them that think without a lause,
With you shall stroke,
Wherefore whone not, I promes of me.

FLOUDEB:
A am now thy; so would sing not see to you these me; with a give my lord, they be grod
They lany.

Third I will you wardion
The like wind and ground are it her and pollow'd with thus snow, but a finen a ceition, if you.

CLEOPATRA:
A made of your fair heartes,
Thou brove thee his man, not grace.

MALVOLIO:
Nay, that fentlement which hime thus though what your hiss obtroud;
To sould wren
Fhole all to fights, we chafferents it.

LADY AF HOHN:
Tell your head; if him. I spolks, sot be procked and
I knefal.

MAISANR:
O, no fursting the confolk
As I do most would the pelive, whose of your grought, let there brow his baster which again, have I these affection a will not grave and green
What make thee, thee my furtent their shall proster and with me to thy stay'd: and the

Epoch: 6/20... Step: 1250... Loss: 1.6381... Val Loss: 1.7494
Dear blesse,
They are the sunther.

CASSAR:
I am ade
To answer.

DUKE OF CAIR:
Sell not swerth that whose they shall place, thought so, her pring his bet on thy
Dot so fair,
As hand in a frusire,
Of thinking a purpory horse that I knater?

DUKE OF YORK:
Strain try
From you that I have have shall not me bady! and the horr, I'll not,
And thought stear you,--O
Till offend with he criscise.

CYRACUS: Herror her-bells.

CLEOPATRA:
Herll te arm
Than, I persance any me had name
With foreriven, sid, my lord, he day.

CLAUDIUS:
He make
But I liven, have, hold by my love
If your have so reman, which hand, then I shall as with him, this heard be as a pray to tuncon anoth, but a greath
My may did!
Phat shopes these they hould at made to your and god,-Oute shise;
Hot somet a prear out and me
As twey'n to part in the which
marray
Their humbiss.

CLEOPATRA:
That who have bear so from the good so his ragk; and dot?

CESTICK:
Be to more
To m

Epoch: 8/20... Step: 1650... Loss: 1.5848... Val Loss: 1.7356
Dear me,
Therefore ever a partunitions this thou has lord;
But this heard.

MARCES:
Alas
The pluckit,
There affecten this hath beer wherefared this: with with a grouch his but on minument a with you deport about to thee we world deat he betret for their hower one to frience, when sence on be but of heart his with for else
When my death, sweet betweep have nob to make mind fair:
Therefant he mout
There's hear of it stars of this hand with had has comeance
To think, with streak and with the poore, and this bowe but as my break
To your gratch own conqueds,
To an expent the word youth to servy:
Ay, and greatness be there,
That were all.

KING PONZESDEL:
Tir, you have tell his dindent thou she butt is heir men, or therefore his soman be fepesh of eveition our stace, you with wook of thy consencans my say your cute herding for his say you so be was we to say; betwend,
And so.

BILAND:
Imen be not speak your callaw is suffolk of st

...

Epoch: 13/20... Step: 2850... Loss: 1.5547... Val Loss: 1.7376
Dear who cassiarmy and terracted hinder:
Ay, iflead, the hand this, have heart,
A am not of with desage; I pind was foung of my rack it walk the will done! lodg:
That eyess
I
lan so for my lord;
Dirst a deed, till blood to tremph.

CHARMILE:
For the grosant thing should being-ban the cinsembland, as I would trud ponfer hang come?

First Gecumporious? we browerlus wear,
Is loss.

ALBXLIO:
Nay your foumbard thy doth,
If them by the life.

DUKE OF YORK:
I hear thee as
Dentler,
And twear
Dot all therefry, and did of hese of seem the fault
To many that our can wondes.

SALARINO:
Hen there's of a master sworn'd and but I shall to dierd of from myself waster?

Second Sut, not sights:
At would have mughters,
That have many all.

CASSICA:
By hell.

BOLENA:
A fuart,
Ay, and say, but I am to strock is thee,
Well youth mades of a goodn her face, my lord and heaven an the do carse army.

Second and to could.

EDAN:
That drunk book
Exmin

Epoch: 15/20... Step: 3250... Loss: 1.6297... Val Loss: 1.7307
Dear at the firm,
Are and I will now west some was weat to that hese offolk shall I was reasung.

DEDEND:
But, might be some but
A freteman a well a peambly, but swren tell his pollows the preshed thy seen with the coppor.
This is fir him
That would of wrat thou warr heep of. To you'las gring a woods,
That yet all to the would
now, I maid be selfots that despronable here it, then she shame for it your sit of the print fearly will prosier and and of meen begon
We she at enceddoms tent be star shall proper'd true to meliample bond,
Ay, but were sissand.

Ploss but I would love-slease wood as thee, I'll scorn, I'll would hands, though of ears,
And more awh,
We nuture's son of her father for help?

SIRALO:
Ay: I hearth'd had with me
So thou sently?

PATES:
Your his bast not!

CASSIUS:
Why, wime thou whold have pity, his:
But I am now, which one,
Hold you will newer;
But him down, but a some to disfear the fire twholss'd him, he

Epoch: 17/20... Step: 3650... Loss: 1.6067... Val Loss: 1.7393
Dearth to thy was store your grace it city, what the chace my made to them your cause to you thee are we muster desin
That ever sad this same true, if east my deel of your may are exe sect be feers which the loved, I will.
Dight, and are such thee, as the greatt.

DEONATO:
I have above with of a sence: your fair of the sable in done of my placest a godly beat you.

BOLAND:
She courtry say.

OCTAVIUS IOFONA:
Why, sir.

BRUTUS:
I hath no cincess me.

HELI:
Here betwee, I'll not to
In hast against mother that's therer of make his gainst with an antlacted and swears this was furtion with fair siffeely and I will are I should to be take the lord, for these army a see, and were a good that's am so.

HIONALD:
They a wust hall'd in your shows'
Soilt.

DOSANIO:
Wherefore that with subjtain and grace the lock'd in thee, and shouldst say your ast me me,
For your a tell, if nothing
They artue, at we art me that is a smen charge of flan

Epoch: 19/20... Step: 4050... Loss: 1.4263... Val Loss: 1.7336
Dear,
But I honows upon me
Home true as makes house.

VIRG IO:
Thou guepune, in thy vares,
And the breatt:
And shall shame.

ANNE PAGE:
Ay, such it they and thy royal my spirious, that your acher
Their twile not sovedal with cannot?

KING JARGAND CAHON:
True wife? Gustice true ill thou have pliffitts,
If the can best,
And come? the carch a suffeds
Bear the cellow of that I mouth,
Bratom up that discopearen with
And bound of me in a prince, and her at
But I seemance,
Nor discriefch; thack murdice; I may I liven one honour hands, it stear
And begom.

PETRUCHIO:
A meant, and be so stand in the would not all see, if will be the part of the heep, I am such no chyiciant will not he bear to be with the plowance
And so bry suse mely lapter
And so true is your coxceases conceile entertainm by thou have ston the grace: brothard would chich me thee.

SIR HUGH EVANS:
Hot shall stall but have baris bodd, he down heard of her.

TALCOLIO:

Epoch: 20/20... Step: 4450... Loss: 1.7257... Val Loss: 1.7359
Dear in hight, the field hath the praised days,
And the chargition day trust to shall not behold in to at is my deed, you las lady add the day
That I say wratues in on hight come would
With mugh my life:
Thou be go,
My lord: I thou what them they as old
Freised and father.
But the swanty, and bay, bears to distables an the pantious might take
As tell,
For a loved
Affly langy of to tell take a kings, with thee;
Therefore for till seem you.

DOHN:
So thoughts again,.

MAADIO:
That shall had bay sit seem the friends and lord;
And let you streat a true, the like it is might, let the sare with a procoment of thou hast for the way brother, my forted with at is dockain!
Do caid her disprocked to coodly.

MENCENTES:
Han worse come, that timen dit of her lady! have a poortul's and biends are thou made be one feads, her father, and shall have of the poor discour thou mind their fears dim with our guilty, and they her face world affie

In [44]:
train(lstm_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/20... Step: 50... Loss: 1.0692... Val Loss: 1.7631
Dear:
Let, I livh for exception, hath not enduct his eye and his own love.

AJAX:
O marry: I'll with dround with you, I will better said
To my between a hair, as you are
the single spirits, and though I fly, them not charge
Stamest were never way in the sight,
And I choked an honour ambition with thee, the brief on Line enemy.

DUKE OF AUMERON:
Sweet me never draw themselves from them to
the subread thou wouldst not been a full on my suffer they seem.

DORFOLK:
I saw they
Brancher I chastions, and many of her come and husband, he was together!

ATHALLO:
Good sorrow when she were deep in fool should you; if I will peffect take it seems as you had not,
To find that it may be to straight been desire then,
He hath not so much her plots? the sacr distry. Thy hath no honour in a miscriun.

BogTTAM:
I thoughts it which are possely?

Second Senator:
That may command they shall not,
I saw you this maidst show for my father, so; where's

Epoch: 3/20... Step: 450... Loss: 1.0176... Val Loss: 1.7705
Dear of thee.

KING JOHN:
Ay, sir, I lay his from my grace of eye then to thee,--

Pirst Huntingman:
Then the worthy hairs of the ground of many won the poor so subjects of a voman.

MALVOLIO:
I curtuness and dreathfull have a fellow, sir, the sea,
To see the garen's son changes
I was a maiden in every throwers to them for the conscusion
On your husband for he?

SICINIUS:
How now, the lose so, for the time what too disclase more a leging of it;
Or, I may will eat their discour.

ANGELO:
Say, I am not tell; him a mist and the deserting of thy prese,
And I between no loyal for thy dost.
And, thou lover from me:
If we'ld make the lord, with court, that are crond, whose arms,
And they must be shalls again.
How now, I thoughts from both.

EXETER:
O pray you, if thou thy dearerty,
The brother bloody mould, they can make
In lament of your mother, madam.

CLEOPATRA:
The slumber to your bloody it:
And show my bring thee, and her prist

Epoch: 4/20... Step: 850... Loss: 1.1402... Val Loss: 1.7811
Dear:
I'll see your love, haste proud, your goodnable me
As due me degutable the poor the cost, the duke: whe sid by way
That it in the craft, which I hath been
to pray them.

CISTOL:
Thou hast from my bount, but they are here,
The man shall not died to bears as we will house,
And where I so bidly his streen's day's her deputy; and I should be man
Would do your life of the gallant stop
The chreeks of the atter, thou will put on me.

LALY MACBETH:
Thou tanded and the flieve his put on my? I am not to be shall follow for your delice;
Unless to the proud to strange heavens, the world, my repomp.

Second Murderer:
There's most confidm.

CLEOPATRA:
He is the little outward and this letter
Dishorous prince the desire, I prople old wife not might sea-thing
And power'd point should hunder, and I am as every tell them, which more pray them her forsing here, my grapple shame.

LOVESS:
Shallow you are live and tongue:
There the world.



Epoch: 6/20... Step: 1250... Loss: 0.9945... Val Loss: 1.7807
Dear
Thanks, poor the plase me, that wrong one office;
If he hath this same contreeds shorts up.

FALSTAFF:
Then to the hollows.

KENT:
I knows to steal you share your parties, o, no fear.

BRUTUS:
She's song: I would have him. Therefore thou hast not dritting short
Hoth her boths of your are never suichin be done, but these that I have her wars,
Who you would not be made
As are power the part for the rhim the shall be man.

MALVOLIO:
Mave no letters. Yourself, she shall reguest hope a case of some might be.

CRESSIDA:
Doth the resudes, and what is so fant stay'd properer
To acted about mine.

DUKE V:
Are now! thy holl of love!

Host:
And lord of the cause.

BRUTUS:
Alas, sir, he shall not well.

ALBAEYES:
My face, and thus forth fall, and now, for the which I listruct,
She is to me yourself to
thy werper;--O is discripted for it.

LADY MACBETH:
Why hath beguiled in him the part aboand about my body sweething bear you
To hav

Epoch: 8/20... Step: 1650... Loss: 0.9599... Val Loss: 1.7995
Dear Forsweet flattery with life;
The preservife would not tarks countain the promises of it: and made
Of which I'll wry my bodim with her dead,
So make him in the fall, and gives of the wusn will dispose twuct without my street! I would be did.

CASSIUS:
I would her way is thou been honourable book; and three, I lie without the pride.

ANNE PAGE:
Why, sweet, have comes no meat,
Whose are he to be so much no heard,
And pride a hubbit told as I should have commend his prince,
I thought it were every teach therefore an I sport:
Provided in my say 'again, as my grave,
In true with sectifies o' my mead more than depuneance
To stroke thee, sir.

CRESSIDA:
O the may would not to pluck was
Hopes under lust be marriage, though to teel; that he sea-day'd and same
And teach to tell;
And the moderation, her blessing no more people's byfall at our other, sir,
If all the weighty with the state,
In love your heart, and bloody heard us,
He

...

Epoch: 13/20... Step: 2850... Loss: 0.9784... Val Loss: 1.8348
Dear pursues affections
For base sight, but what is ready house;
For well as not court to you, but you take thy but
He hath heard him, 'tis racabuse and true me with
A gentlemen, and then to honour, sir, I am a garment; shall I most be their honour practise
Him how be so my lord.

DUKE VINCENTIO:
Sir, if you are to my ranks.

SPEED:
These monder, and the who were never that face.

Boatswain:
Why, that's it sooner:
But made a man won not an honour, and therefore, sir: I will:
And power stone, he that drunk of day discourse so ranch, sometimes
Is fell all do as on your brother, and pluck doth to seem a trumpet practise in fire;
The observant, thy deserts for them.

KING HENRY V:
Come an eyes at the name,
But was of my hands.
Prom my sword of means.

Second Citizen:
A hurt thou art thee will be stucd begun.

ANBETH:
It was gain his day's hold him sweet should
Go pass upon to my draw?

TIMON:
Decond Men and have not page,
Sound

Epoch: 15/20... Step: 3250... Loss: 0.9164... Val Loss: 1.8525
Dear
As you wilt actiffor thy request so man,
And seem a man since thy reasonable see marking stuff
As you knows to praised in the cold bald did they well.

KENT:
Po, that it! Go, whose arm such a friends
My heart, madam!

First Yorfolk would I prevail'd it,
Contemperitage and the said deliver.

DUKE VINCENTIO:
Now ha! now, say you shall friends and ground of this back,
So would be more offence they was a man is bloody know but from the face, you have his son.

CASSIUS:
We'll noils a fair wonder and from me in still ever throw thee
And shall so marbosed to make me by-ecains upon them, made;
And subson down then be no houchs hath doth.

AETALIX:
By my troth, her fortune,
How now, Peace.

BASSANIO:
I think, we spokes with the longem, and meloughts.

QUEENREN:
And what thou will, as I have make no duty are bear here;
And be with a good Margaret make her plotanished till day
As well deputy is well enough.

Second Murderer:
Then

Epoch: 17/20... Step: 3650... Loss: 0.9482... Val Loss: 1.8729
Dear
That dulish because so I will enter.

OTHELLO:
Why, stay, they shall make the heart
With show him alone,
Let's a miscriffer's song,
And man, I was sisted's hand being overfain with his earling performance
To be more than that he does,
Trongs up all my mine eyes to my farth their father's take my wife?

BRUTUS:
Where's a man exhand am I should I say yet I would
to dust not constore any treasure
Whose can absence I would not tend those this.

BERCEMIAS:
Why, that is the hands and many as my scorn,
Who must not blessing of it cur of thy shall few shall
Be sit weep you in them.

SPEED:
I'll fighty peace to my fortune hath thy face I did not light
With the stroke might be, I will go, But to the worsuch and alsomething battle,
Who his rich above, the dear an excellent country, therefore, 'tis not with your daughter give
The say whose aldit, I so still of their fity
And my father and for will chamber the windred have been wit

Epoch: 19/20... Step: 4050... Loss: 0.8326... Val Loss: 1.8855
Dear:
The love, that kingdom but that feed
That I should be speech begin. They are all and strange my streets in the expreath of heavens.

BASSANIO:
He is but so but here comes love by dispance.

NORFOLK:
Tell, my lord.

VALK:
I can'st the names, and, the mother their rever to you; but any master and honourable on Lord:
When I will pair with contreatening out of to that the solemnized against bloody eyes,
Sing compannel let's heard in the kingdom.

CHARMIAN:
So behind thy hand:
She will be weephy time of this sweet conspirity;
And descripted in your charge.
What holio, good tilm but the sight, and thee:--

CINNA:
A harm not done, and be so friends to have show my praised together, high regreet brought me truly bloody a write.

KING RICHARD III:
Valiant show me that how not?

CLEOPATRA:
Wear my give it thy hossess'd, and thus just and fair thee:
There's like to rust in the flace of expedition hands;
I do not, but if it thus,

Epoch: 20/20... Step: 4450... Loss: 0.9910... Val Loss: 1.8998
Dear:
But tell you the spirit as sea may which was daughter say
And delivery such a fair corrected my friend traze
To have in thy fool and light,
Or else seem'd him gone.

BRUTUS:
My lord, you shall fall, and for when the hay the nexemberiagh,
But where I do not see that, and straight way say a trath still: and besom without discrot of your life, you hath was shorts,
And when I would show as same chain's entreated, I will be some that trumpet, or suffers;
How now, a tretch'd in some life, my grave but like,
To steal that same citch of Corrow, poor country's thusbain, and,
For you do abace, what a sister court.

Bust ANtony:
But star''s trust them at our siffacies all observant.

MENENIUS:
Cares, his convenchers,
But thou must be so melication could,
And beauty of the whole unconsent, and there but the chain the guilty age, aim and what is this
Dust fents so, lady! you what desired to see it,
Of thou ark my bear the single h

##### Generate text

In [45]:
print(sample(vanilla_model, 500, prime='The', top_k=10))

These with were
To thy have the was
Sere in the genmit host she hand outlive
The grace, and so you much as hy's thus of which not thee.

DUKE VINCENTIO:
Where in the gueshy thy be sinder
Lo, was it a fornit, and stur sheeld hath not on, shall hatun as you?

DOSALINES:
O as I this.

MIANA:
I would shall been what you crant, have the grepare that doubs afain that your bodies it,
My lord; I am but in myself. Your sit of to contemplings you?

ANNE PAGE:
Sir, them.

MARK ANTONY:
What shear the was anoth'


In [46]:
print(sample(lstm_model, 500, prime='The', top_k=10))

The pawnant, might come, they have break the dear his bed:
And make thee then with my sword.

DUKE VINCENTIO:
Sir, you she should have here, inless something his chaft,
She should I know me the chamber
And we shall be such guilty.

QUEEN MARGARET:
What master so bring suffer me my boy; I'll catt:
I would not to will pay my sword, my lord.

VENTIDIUS:
Nor not of this bloody:
Vour give him country tongue brother's hour thy father of the pleasure,
And blind, my good lordship's changed his grace is not 


## Results:
- Changing the number of layers: 5 layers just produced nonsense text, maybe the higher number of layers would need significantly higher number of training epochs. 2 layers seem to be fine with 20 epochs and produce almost readable text.
- Changing the sequence length: short sequence lengths like 1 produce random signs and more nonesense letters and almost bno dialogue, higher lengths produce more dialogue and better sentence structure, too long looses meaningfull words
- Changing the batch size: no huge differences
- 2 layers, batchsize 20 and sequence length 20 was, with the empirically chosen values, the best result
- there does not seem to be a huge difference between the vanilla RNN and the LSTM in this example